

```
Target:
        1) Have to achive the given target accuracy and should be consistent.
        2) Use different Image Augmantation and different LR scheduler.
        3) Only 15 epochs.
Result:
        1) Best Training Accuracy: 98.69
        2) Best Test Accuracy: 99.45
        3) Params: 9.98k.
Analysis:
        1) i got what is required.
        2) LR scheduler helped because i have seen there was accuracy stablising at around last few epochs, so i should
        lower the LR over there.
        3) This model can be further pushed but not required as of now :-)

```



In [9]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [10]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.convolution1 = nn.Sequential(
        nn.Conv2d(1, 10, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(10),
        nn.Conv2d(10, 20, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(20),
        nn.MaxPool2d(2,2),
        nn.Dropout(0.15)
    )
    self.convolution2 = nn.Sequential(
        nn.Conv2d(20, 16, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(16),
        nn.Conv2d(16, 12, 3, padding=1),
        nn.ReLU(),
        nn.BatchNorm2d(12),
        nn.MaxPool2d(2,2),
        nn.Dropout(0.15)
    )
    self.convolution3 = nn.Sequential(
        nn.Conv2d(12, 12, 3),
        nn.Conv2d(12, 10, 3),
        nn.Conv2d(10, 10, 3)
        # nn.ReLU(),
        # nn.Conv2d(16, 10, 3),
    )
  def forward(self, x):
    x = self.convolution1(x)
    x = self.convolution2(x)
    x = self.convolution3(x)
    x = x.view(-1,10)
    return F.log_softmax(x)

In [11]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
              ReLU-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
            Conv2d-4           [-1, 20, 28, 28]           1,820
              ReLU-5           [-1, 20, 28, 28]               0
       BatchNorm2d-6           [-1, 20, 28, 28]              40
         MaxPool2d-7           [-1, 20, 14, 14]               0
           Dropout-8           [-1, 20, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           2,896
             ReLU-10           [-1, 16, 14, 14]               0
      BatchNorm2d-11           [-1, 16, 14, 14]              32
           Conv2d-12           [-1, 12, 14, 14]           1,740
             ReLU-13           [-1, 12, 14, 14]               0
      BatchNorm2d-14           [-1, 12,

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [12]:
torch.manual_seed(1)
batch_size=128
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
                datasets.MNIST("../data", train=True, download=True,
                               transform = transforms.Compose([
                                            transforms.RandomAffine(degrees=3, translate=(0.1,0.1), scale=(0.9, 1.1)),
                                            #transforms.ColorJitter(brightness=0.1, contrast=0.2),
                                            transforms.ToTensor(),
                                            transforms.Normalize((0.1307,), (0.3081,))
                               ])),
                batch_size=batch_size, shuffle=True, **kwargs
              )
test_loader = torch.utils.data.DataLoader(
              datasets.MNIST("../data", train=False, download=True,
                             transform = transforms.Compose([
                                          transforms.ToTensor(),
                                          transforms.Normalize((0.1307,),(0.3081,))
                             ])),
              batch_size=batch_size, shuffle=True, **kwargs
              )

In [23]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,13], gamma=0.2)

In [24]:
from tqdm import tqdm
train_loss = []
train_acc = []
def train(model, device, train_loader, optimizer, epoch):
    model.train()

    correct=0
    processed=0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        train_loss.append(loss)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)

        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Accuracy={100*correct/processed}')
        train_acc.append(100*correct/processed)
    scheduler.step()


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [25]:
for epoch in range(0, 15):
  print (f"epoch : {epoch}")
  train(model, device, train_loader, optimizer, epoch)
  test(model, device, test_loader)

epoch : 0


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
loss=0.15729832649230957 batch_id=468 Accuracy=88.605: 100%|██████████| 469/469 [00:34<00:00, 13.65it/s]



Test set: Average loss: 0.0456, Accuracy: 9848/10000 (98%)

epoch : 1


loss=0.22730036079883575 batch_id=468 Accuracy=96.53166666666667: 100%|██████████| 469/469 [00:34<00:00, 13.69it/s]



Test set: Average loss: 0.0464, Accuracy: 9849/10000 (98%)

epoch : 2


...


Test set: Average loss: 0.0185, Accuracy: 9942/10000 (99%)

epoch : 13


loss=0.16691547632217407 batch_id=468 Accuracy=98.62833333333333: 100%|██████████| 469/469 [00:34<00:00, 13.72it/s]



Test set: Average loss: 0.0188, Accuracy: 9943/10000 (99%)

epoch : 14


loss=0.01650872640311718 batch_id=468 Accuracy=98.67666666666666: 100%|██████████| 469/469 [00:33<00:00, 13.80it/s]



Test set: Average loss: 0.0182, Accuracy: 9945/10000 (99%)

